In [1]:
from script import df_mobpro_brut, arr_marseille, arr_paris, arr_lyon, contours_comm, transport_dict, contours_comm, flux
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.colors as colors
from scipy.sparse import coo_matrix

/home/onyxia/work/statapp/script.py:19: DtypeWarning: Columns (1,1687,1688) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dossier_complet_brut = pd.read_csv(file)
/home/onyxia/work/statapp/script.py:24: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta_dossier_complet_brut = pd.read_csv(file)
/home/onyxia/work/statapp/script.py:34: DtypeWarning: Columns (1,2,3,4,14,15,16,18,22,24,27,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mobpro_brut = pd.read_csv(file)


KeyboardInterrupt: 

In [ ]:
df_mobpro_brut.head()

Ce que signifient chacune des en-têtes de colonnes :  
COMMUNE Département et commune du lieu de résidence  
ARM Arrondissement municipal de résidence (Paris, Lyon et Marseille)  
DCFLT Commune frontalière du lieu de travail  
DCLT Département, commune et arrondissement municipal (Paris, Lyon, Marseille) du lieu de travail  
AGEREVQ Âge quinquennal en années révolues  
CS1 Catégorie socioprofessionnelle en 8 postes  
DEROU Nombre de deux-roues à moteur du ménage (DOM)  
DIPL_15 Diplôme le plus élevé  
EMPL Condition d'emploi  
ILTUU Indicateur urbain du lieu de travail  
ILT Indicateur du lieu de travail  
IMMI Situation quant à l'immigration  
INATC Indicateur de nationalité condensé (Français/Étranger)  
INEEM Nombre d'élèves, étudiants ou stagiaires âgés de 14 ans ou plus du ménage  
INPOM Nombre de personnes actives ayant un emploi du ménage  
INPSM Nombre de personnes scolarisées du ménage  
IPONDI Poids de l'individu  
LPRM Lien à la personne de référence du ménage  
METRODOM Indicateur Métropole ou DOM du lieu de résidence  
MOCO Mode de cohabitation  
NA5 Activité économique regroupée en 5 postes  
NPERR Nombre de personnes du ménage (regroupé)  
REGION Région du lieu de résidence  
REGLT Région du lieu de travail  
SEXE Sexe  
STAT Statut professionnel  
STOCD Statut d'occupation détaillé du logement  
TP Temps de travail  
TRANS Mode de transport principal le plus souvent utilisé pour aller travailler  
TYPL Type de logement  
TYPMR Type de ménage regroupé (en 9 postes)  
VOIT Nombre de voitures du ménage  

In [ ]:
# Distribution des modes de transport utilisés pour aller travailler
transport_distribution = df_mobpro_brut['TRANS_LABEL'].value_counts()
print(transport_distribution)

# Visualisation
transport_distribution.plot(kind='bar', color='skyblue')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.title('Distribution des modes de transport')
plt.xlabel('Mode de transport')
plt.ylabel('Nombre d’individus')
plt.show()

In [ ]:
# Âge moyen par mode de transport
age_transport = df_mobpro_brut.groupby('TRANS_LABEL')['AGEREVQ'].mean()
print(age_transport)

# Visualisation
age_transport.plot(kind='bar', color='orange')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.title('Âge moyen par mode de transport')
plt.xlabel('Mode de transport')
plt.ylabel('Âge moyen')
plt.show()


In [ ]:
# Distribution des modes de transport par catégorie socio-professionnelle avec labels
cs1_transport = pd.crosstab(df_mobpro_brut['CS_LABEL'], df_mobpro_brut['TRANS_LABEL'])

# Heatmap
sns.heatmap(cs1_transport, annot=True, fmt='d', cmap='Blues')
plt.title('Modes de transport par catégorie socio-professionnelle')
plt.xlabel('Mode de transport')

# Inclinaison des labels avec alignement pour que l'extrémité pointe vers l'axe
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.ylabel('Catégorie socio-professionnelle')
plt.show()


La fonction ci-dessous permet d'obtenir un premier tableau regroupant certaines informations par commune :
- Nombre de personnes du ménage (regroupé)
- Nombre de personnes scolarisées du ménage
- Nombre de personnes actives ayant un emploi du ménage
- Nombre d'élèves, étudiants ou stagiaires âgés de 14 ans ou plus du ménage
- Indicateur du lieu de travail
(1 Dans la commune de résidence actuelle
2 Dans une autre commune du département de résidence
3 Dans un autre département de la région de résidence
4 Hors de la région de résidence actuelle : en métropole
5 Hors de la région de résidence actuelle : dans un DOM
6 Hors de la région de résidence actuelle : dans une COM
7 A l'étranger)

In [ ]:
# Conversion des colonnes en numérique
df_mobpro_brut['NPERR'] = pd.to_numeric(df_mobpro_brut['NPERR'], errors='coerce')
df_mobpro_brut['INPSM'] = pd.to_numeric(df_mobpro_brut['INPSM'], errors='coerce')
df_mobpro_brut['INPOM'] = pd.to_numeric(df_mobpro_brut['INPOM'], errors='coerce')
df_mobpro_brut['INEEM'] = pd.to_numeric(df_mobpro_brut['INEEM'], errors='coerce')

# Création des agrégations de base
mobpro_villes = df_mobpro_brut.groupby('COMMUNE').agg({
    'NPERR': 'sum',
    'INPSM': 'sum',
    'INPOM': 'sum',
    'INEEM': 'sum'})

# Créons d'abord un DataFrame temporaire pour chaque valeur d'ILT
for i in range(1, 8):
    # Grouper par COMMUNE et compter les occurrences où ILT == str(i)
    temp_count = df_mobpro_brut[df_mobpro_brut['ILT'] == i].groupby('COMMUNE').size()
    # Ajouter cette série au DataFrame principal
    mobpro_villes[f'ILT_{i}'] = temp_count

# Remplir les valeurs manquantes (NaN) par 0
mobpro_villes = mobpro_villes.fillna(0)

# Réinitialisation de l'index
mobpro_villes = mobpro_villes.reset_index()

In [ ]:
mobpro_villes.head()

On souhaite créer une matrice des flux pour avoir une vision générale des trajets domicile-travail quotidiens entre les villes. Pour chaque couple de ville, le nombre correspondant dans la matrice correspond au nombre de commute effectué de la ville 1 à la ville 2. 

On remarque que pour la variable "COMMUNE" (commune de résidence) on a uniquement 75056 pour Paris (idem pour Lyon et Marseille), tandis que pour la variable "DCLT" (commune de travail) on a un découpage en arrondissement avec les codes 75101,... pour Paris (idem pour Lyon et Marseille). Afin de pallier cette différence, on se cale sur l'échelle des arrondissements en remplaçant la valeur de "COMMUNE" par celle de "ARM" lorsque la ville en question est Paris, Marseille ou Lyon.

In [ ]:
# Création d'une table croisée dynamique pour compter les déplacements
flux_tot = pd.crosstab(df_mobpro_brut['COMMUNE'], df_mobpro_brut['DCLT'])

# Conversion en DataFrame pour plus de clarté
flux_tot = pd.DataFrame(flux_tot)


# Pour voir les dimensions de la matrice
print("Dimensions de la matrice :", flux_tot.shape)

# Filtrer les données pour chaque groupe de transport
df_trans_45 = df_mobpro_brut[df_mobpro_brut['TRANS'].isin([4, 5])]
df_trans_6 = df_mobpro_brut[df_mobpro_brut['TRANS'] == 6]
df_trans_123 = df_mobpro_brut[df_mobpro_brut['TRANS'].isin([1, 2, 3])]

# Créer les matrices des flux pour chaque groupe
flux_rouge = pd.crosstab(df_trans_45['COMMUNE'], df_trans_45['DCLT'])
flux_jaune = pd.crosstab(df_trans_6['COMMUNE'], df_trans_6['DCLT'])
flux_vert = pd.crosstab(df_trans_123['COMMUNE'], df_trans_123['DCLT'])

# Convertir en DataFrame pour plus de clarté
flux_rouge = pd.DataFrame(flux_rouge)
flux_jaune = pd.DataFrame(flux_jaune)
flux_vert = pd.DataFrame(flux_vert)

# Afficher les dimensions des matrices
print("Dimensions de la matrice (TRANS = 4 ou 5) :", flux_rouge.shape)
print("Dimensions de la matrice (TRANS = 6) :", flux_jaune.shape)
print("Dimensions de la matrice (TRANS = 1, 2 ou 3) :", flux_vert.shape)


In [ ]:
flux('78220', '75108')

In [ ]:
flux_rouge

In [ ]:
# Pour les départs : somme sur l'axe des colonnes (chaque ligne = ville de départ)
df_flux_jaune_depart = flux_jaune.sum(axis=1).reset_index()
df_flux_jaune_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_jaune_destination = flux_jaune.sum(axis=0).reset_index()
df_flux_jaune_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_jaune_depart.head())
print("Destination :", df_flux_jaune_destination.head())

df_flux_vert_depart = flux_vert.sum(axis=1).reset_index()
df_flux_vert_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_vert_destination = flux_vert.sum(axis=0).reset_index()
df_flux_vert_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_vert_depart.head())
print("Destination :", df_flux_vert_destination.head())

df_flux_rouge_depart = flux_rouge.sum(axis=1).reset_index()
df_flux_rouge_depart.columns = ['COMMUNE', 'flux_depart']

# Pour les arrivées : somme sur l'axe des lignes (chaque colonne = ville d'arrivée)
df_flux_rouge_destination = flux_rouge.sum(axis=0).reset_index()
df_flux_rouge_destination.columns = ['DCLT', 'flux_destination']

print("Départ :", df_flux_rouge_depart.head())
print("Destination :", df_flux_rouge_destination.head())


In [33]:
df_flux_jaune_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_jaune_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_jaune_depart_m["flux_depart"] = df_flux_jaune_depart_m["flux_depart"].fillna(0).astype(int)

df_flux_vert_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_vert_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_vert_depart_m["flux_depart"] = df_flux_vert_depart_m["flux_depart"].fillna(0).astype(int)

df_flux_rouge_depart_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_rouge_depart,
        left_on='INSEE_COM',
        right_on="COMMUNE",
        how='left')
df_flux_rouge_depart_m["flux_depart"] = df_flux_rouge_depart_m["flux_depart"].fillna(0).astype(int)


df_flux_jaune_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_jaune_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_jaune_destination_m["flux_destination"] = df_flux_jaune_destination_m["flux_destination"].fillna(0).astype(int)

df_flux_vert_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_vert_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_vert_destination_m["flux_destination"] = df_flux_vert_destination_m["flux_destination"].fillna(0).astype(int)

df_flux_rouge_destination_m = pd.merge(
        contours_comm[['INSEE_COM', 'NOM', 'POPULATION', 'INSEE_DEP', 'SIREN_EPCI', 'geometry']],
        df_flux_rouge_destination,
        left_on='INSEE_COM',
        right_on="DCLT",
        how='left')
df_flux_rouge_destination_m["flux_destination"] = df_flux_rouge_destination_m["flux_destination"].fillna(0).astype(int)

In [ ]:
plot_flux_gradient(df_flux_rouge_depart_m, "rouge", "Figure - Flux (domicile-travail) de départ \n en voiture/moto par commune","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_jaune_depart_m, "jaune", "Figure - Flux (domicile-travail) de départ \n en transport en commun par commune","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_vert_depart_m, "vert", "Figure - Flux (domicile-travail) de départ \n à pied","flux_depart")

In [ ]:
plot_flux_gradient(df_flux_rouge_destination_m, "rouge", "Figure - Flux (domicile-travail) d'arrivée \n en voiture/moto","flux_destination")

In [ ]:
plot_flux_gradient(df_flux_jaune_destination_m, "jaune", "Figure - Flux (domicile-travail) d'arrivée \n en transport en commun","flux_destination")

In [ ]:
plot_flux_gradient(df_flux_vert_destination_m, "vert", "Figure - Flux (domicile-travail) d'arrivée \n à pied","flux_destination")